In [1]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Sequence
from sqlalchemy import Column, Integer, Float, Boolean, String
from sqlalchemy.orm import sessionmaker

In [2]:
import time

In [6]:
engine = create_engine('sqlite:///:memory:', echo=False)
BASE = declarative_base()
BASE.metadata.create_all(engine)
session = sessionmaker(bind=engine)()

In [9]:
def Read_TemperatureSensor1():
    class TemperatureSensor1(BASE):
        __tablename__ = 'temperature sensor 1'
        id = Column(Integer, Sequence('TemperatureSensor1_id_seq'), primary_key=True)
        time = Column(Float) # seconds since the Epoch
        temperature = Column(Float) # centigrade
        
        def __repr__(self):
            return "<TemperatureSensor1(time='%f', temperature='%f')>" % (self.time, self.temperature) 
    while True:
        T=-999.999 # get temperature in centigrade
        update = TemperatureSensor1(time=time.time(), temperature=T)
        session.add(update)
        session.commit()
        #now = session.query(TemperatureSensor1).order_by(TemperatureSensor1.id.desc()).first()
        #now
        #now.time
        #now.temperature
        #for instance in session.query(TemperatureSensor1).order_by(TemperatureSensor1.id.desc()):
            #print instance.time, instance.temperature  

In [10]:
def Read_Heater1():
    class Heater1(BASE):
        __tablename__ = 'heater 1'
        id = Column(Integer, Sequence('Heater1_id_seq'), primary_key=True)
        time = Column(Float) # seconds since the Epoch
        on_state = Column(Boolean) # True is on, False is off
        
        def __repr__(self):
            return "<Heater1(time='%f', on_state='%s')>" %(self.time, str(self.on_state)) 
    while True:
        on = True # get temperature in centigrade
        update = Heater1(time= time.time(), on_state = on)
        session.add(update)
        session.commit()
        #now = session.query(Heater1).order_by(Heater1.id.desc()).first()
        #now
        #now.time
        #now.temperature
        #for instance in session.query(Heater1).order_by(Heater1.id.desc()):
            #print instance.time, instance.temperature  